In [ ]:
!pip install --upgrade xlrd

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.integrate import quad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/Blackcoffer_Files/Datasets/DATA2/DATA_Experiment3/42.xls',sheet_name='results',header=None)
df

In [ ]:
def f1(sigma,t):   # when we use this function, t will go from 1 till 300 (not 0-299)
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):  # we want it to go from start ... t, that's why used t+1                  
    i = k-1  # id of data point, since k is id+1 (k is in column no.0 which is id+1)
    val = math.exp(-df[1][i]*df[1][i] / (2*sigma*sigma))
    val = val / (math.sqrt(2*math.pi)*sigma)
    result = result * val
  return result

In [ ]:
def f2(t):
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):                                   # 
    i = k-1
    val = math.pi*(1 + df[1][i]*df[1][i])
    val = 1 / val
    result = result * val
  return result

In [ ]:
def m1(t):
  val = lambda t : quad(f1, 0.1,2, args=(t,))
  ans,err = val(t)
  return (1/1.9)*ans

In [ ]:
def m2(t):
  return f2(t)

In [ ]:
Pm1 = 0.5
Pm2 = 0.5

In [ ]:
def Pm1Xt(t):
  val = Pm1 * m1(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [ ]:
def Pm2Xt(t):
  val = Pm2 * m2(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [ ]:
def sigmaapprox(t):
  start = 20*math.floor((t-1)/20)+1                                                 
  end = t
  if start==t: 
    return 0  # no variance if there is only one data point
  result = 1 / (end-start+1)
  val = 0
  for k in range(start,end+1):
    i = k-1
    val += df[1][i]*df[1][i]
  result = result * val
  result = math.sqrt(result)
  
  return result

In [ ]:
def pg(t):
  sigma = sigmaapprox(t)
  if sigma==0:
    return 0
  def integrand(x):
      result = 1 / (sigma*math.sqrt(2*math.pi))
      result = result * math.exp(-x*x / (2*sigma*sigma))
      return result
  phi,err = quad(integrand,-float('inf'),4)
  result = 1 - 2*(1-phi)
  
  return result  

In [ ]:
def pc():
  return 0.844

In [ ]:
def Cfunc(theta,t):
  C = 0
  for i in range(0,t%20):
    C+= pow(theta,i)
  return C

In [ ]:
def DAfunc(kappa1 , kappa2 , theta ):
  DA_list =[]
  C = 0
  for i in range(0,20):
    if i == 0:
      DA_list.append(0)
      continue
    C = Cfunc(theta,i)
    DA_list.append(round(max(0, -kappa2 + 1/(1+math.exp(-kappa1*C))),3))
  return DA_list

In [ ]:
def compute_expValue(DA, t):
  if t%20==0 or t%20==1: return 0   # t is the first or the last data point of a trial set
  Pg = pg(t)
  Pc = pc()
  V = Pm1Xt(t-1)*((2+DA)*Pg - (1-Pg)*40) + Pm2Xt(t-1)*((2+DA)*Pc-(1-Pc)*40)
  return V # expected value

We bet on trial t+1 if and only if Vt > 0

In [ ]:
def predict(start,end,DA,beta):
  total = 0
  plist = []
  #explist = []
  probBaseList = []
  probList = []
  for i in range(start,end):
    # i is the time stamp
    Vt = compute_expValue(DA[i%20],i)
    #explist.append(Vt)
    prob_base = math.exp(beta*Vt) / (1 + math.exp(beta*Vt))
    prob = DA[i%20]+(1-DA[i%20])*prob_base
    probBaseList.append(prob_base)
    probList.append(prob)
    if prob>0.5:
      plist.append(1)
      if df[1][i]<=4 and df[1][i]>=-4: 
        total+=2
      else: 
        total-=40
    else: plist.append(0)        
  return plist,probBaseList,probList,total

Now we train the model (the free parameters kappa1, kappa2, beta)

In [ ]:
Y = df[5]#df[4] for samples 40-124-Experiment1

In [ ]:
def gradient_cost_func(kappa1,kappa2,theta,beta):
  grad_k1 = 0
  grad_k2 = 0
  grad_beta = 0
  DA = DAfunc(kappa1,kappa2,theta)
  plist,probBaseList,probList,total = predict(0,160,DA,beta)
  for i in range(3,160):
    C = Cfunc(theta,i)
    if i%20<5: continue
    grad_k1 += ((probList[i]-Y[i])*((C*math.exp(-kappa1*C))/((1+math.exp(-kappa1*C))*(1+math.exp(-kappa1*C)))))/(probList[i]*(1-probList[i]))*(1-probBaseList[i])
    grad_k2 += ((probList[i]-Y[i])*(-1))/(probList[i]*(1-probList[i]))*(1-probBaseList[i])
    grad_beta += (probList[i]-Y[i])/(probList[i]*(1-probList[i]))*(1-DA[i%20])*probBaseList[i]*(1-probBaseList[i])*compute_expValue(DA[i%20],i)
  
  grad_k1/=160 
  grad_k2/=160
  grad_beta/=160 
  return (grad_k1, grad_k2,grad_beta)

In [ ]:
kappa1 = 0.5
kappa2 = 0.5
theta = 0.9
beta = 1
learning_rate = 0.1  # learning rate
i = 0
while True:
  if i==100: break
  
  if i==0:
    pre_k1,pre_k2,pre_beta = kappa1,kappa2,beta
    i+=1
  else:
    G_k1,G_k2,grad_beta = gradient_cost_func(kappa1,kappa2,theta,beta)
    temp1 = kappa1 - learning_rate*G_k1
    temp2 = kappa2 - learning_rate*G_k2
    temp3 = beta - learning_rate*grad_beta
    kappa1,kappa2,beta = temp1,temp2,temp3
    if kappa1>1: kappa1 = 0.99
    if kappa1<0.1: kappa1 = 0.1
    if kappa2<0.2: kappa2 = 0.2
    if kappa2 > 1: 
      kappa2 = 0.99
      break
    if beta<0: beta = 0.1
    if abs(kappa2-pre_k2)<1e-3 and abs(beta-pre_beta)<1e-3: break
    else: 
      pre_beta = beta
      pre_k2 = kappa2
    print(str(i)+". kappa1, kappa2, beta: ",kappa1,kappa2,beta)
    i+=1    

print("kappa1, kappa2, beta: ",kappa1,kappa2,beta)

In [ ]:
plist,probBaseList,probList,total = predict(160,300,DAfunc(kappa1, 0.8, theta),beta)
print(len(plist))
print(total)

In [ ]:
total = 0
for i in range(0,140):
  if (plist[i]==0 and Y[i+160]==0) or (plist[i]==1 and Y[i+160]==1): total+=1
print("The accuracy score of the model is: ",total/140*100,"%")